In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [6]:

import langchain
print(f"LangChain version: {langchain.__version__}")

LangChain version: 1.0.0a12


# create_agent

In [8]:
from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model="anthropic:claude-3-7-sonnet-latest",
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='8fb520be-1008-442b-a915-8d57facaf2e8'),
  AIMessage(content=[{'text': 'I can help you check the weather in San Francisco. Let me get that information for you.', 'type': 'text'}, {'id': 'toolu_01G5R1qkXz6f2dfuUfzayPVA', 'input': {'city': 'San Francisco'}, 'name': 'get_weather', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01A9sCJ8UUfN4te3npX9xTo5', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 386, 'output_tokens': 74, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-7-sonnet-20250219'}, id='lc_run--49144a5c-4a20-41c6-870c-a09375a186bb-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'

## Additional fields in inputs renders weird 
- Doesnt have the "Additional fields" callout like the output does
- Shows up BEFORE the list of messages (should be after)

# init_chat_model

In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-4.1")
llm.invoke("Why do parrots talk?")


AIMessage(content='Parrots "talk" because they are exceptional mimics, able to imitate a wide variety of sounds—including human speech. But *why* do they do this?\n\n### 1. **Social Nature**\nParrots are highly social animals in the wild. Their vocal abilities help them communicate complex messages with their flock—letting others know about food sources, danger, or simply maintaining social bonds. In captivity, humans become their flock, so they try to "fit in" by copying the sounds they hear most often—our speech.\n\n### 2. **Intelligence**\nParrots are among the most intelligent birds, able to recognize, remember, and recreate sounds. They don’t just randomly repeat noise; in some cases, they use appropriate words or phrases in context, showing a kind of understanding or association.\n\n### 3. **Attention and Interaction**\nParrots quickly learn that making certain sounds (like words) earns them attention, treats, or playtime from their human companions. Mimicking speech thus becomes

In [14]:
inputs = {
  "messages": [
    {
      "id": "msg_123",
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What breed is this dog?"
        },
        {
          "type": "image",
          "source_type": "url",
          "url": "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U",
          "mime_type": "image/jpeg"
        }
      ]
    }
  ]
}

llm.invoke(inputs["messages"])


AIMessage(content='This dog appears to be a **Black Labrador Retriever** puppy. Black Labs are known for their friendly and gentle nature, as well as their shiny black coat, floppy ears, and expressive eyes. They are one of the most popular dog breeds around the world.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 268, 'total_tokens': 320, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_d1d5c01587', 'id': 'chatcmpl-COXNfOyjThf6rqftQH5XPkLsdUBe5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--b2c2f6bc-a74c-4f16-8836-61d4b8d8b343-0', usage_metadata={'input_tokens': 268, 'output_tokens': 52, 'total_tokens': 320, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'o

# model_with_tools

In [20]:
from langchain_core.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    return f"It's sunny in {location}."


model_with_tools = llm.bind_tools([get_weather])

response = model_with_tools.invoke("What's the weather like in Boston?")

# structured outputs

In [26]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = llm.with_structured_output(Movie, include_raw=True)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # Movie(title="Inception", year=2010, director="Christopher Nolan", rating=8.8)

{'raw': AIMessage(content='{"title":"Inception","year":2010,"director":"Christopher Nolan","rating":8.8}', additional_kwargs={'parsed': Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8), 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 126, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_d1d5c01587', 'id': 'chatcmpl-COXeTEp0N6GQP8pPp22MGAKJ2kFZh', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--59884d3a-ed3a-47d5-8011-3782be83b292-0', usage_metadata={'input_tokens': 126, 'output_tokens': 22, 'total_tokens': 148, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), 'parsed': Movie(title=

# tools and tool messages

In [7]:
from langchain.messages import ToolMessage


# After a model makes a tool call
ai_message = AIMessage(
    content=[],
    tool_calls=[{
        "name": "get_weather",
        "args": {"location": "San Francisco"},
        "id": "call_123"
    }]
)

# Execute tool and create result message
weather_result = "Sunny, 72°F"
tool_message = ToolMessage(
    content=weather_result,
    tool_call_id="call_123"  # Must match the call ID
)

# Continue conversation
messages = [
    HumanMessage("What's the weather in San Francisco?"),
    ai_message,  # Model's tool call
    tool_message,  # Tool execution result
]
response = model.invoke(messages)  # Model processes the result

# Content Blocks 

In [17]:
from langchain_core.messages import HumanMessage

# String content
messages = [HumanMessage(content_blocks=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image", "url": "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U"},
])] 

llm.invoke(messages)

AIMessage(content="Hello! I'm doing well, thank you. How about you? \n\nBy the way, what an adorable black puppy in your photo! Is this your dog?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 268, 'total_tokens': 300, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_e24a1fec47', 'id': 'chatcmpl-COrh9aSmK7P4Haa8l3IV3TD80j9Mj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--f7925c90-3e97-4130-93b5-5551ac196ee0-0', usage_metadata={'input_tokens': 268, 'output_tokens': 32, 'total_tokens': 300, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [22]:
from langchain_core.messages import AIMessage
from langsmith import traceable


inputs = [HumanMessage(content_blocks=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image", "url": "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U"},
])] 


output_msg = AIMessage(
    content=[
        {"type": "thinking", "thinking": "...", "signature": "WaUjzkyp..."},
        {"type": "text", "text": "..."},
    ],
    response_metadata={"model_provider": "anthropic"}
)


outputs = output_msg.content_blocks


@traceable(run_type="llm")
def llm_raw(inputs):
    return outputs


llm_raw(inputs)




[{'type': 'reasoning',
  'reasoning': '...',
  'extras': {'signature': 'WaUjzkyp...'}},
 {'type': 'text', 'text': '...'}]

## Content blocks rendering

I think there's a gap in how we render content blocks: https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&peek=bdb7da3d-14ef-4257-aa8d-35ab9becc8a7&peeked_trace=bdb7da3d-14ef-4257-aa8d-35ab9becc8a7

# Multi-modal

In [ ]:
#images

import base64
import httpx

image_url = "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U"
image_base64 = base64.standard_b64encode(httpx.get(image_url).content).decode("utf-8")


input = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {"type": "image", "url": "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U"},
    ]
}

# From base64 data
output = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {
            "type": "image",
            "base64": image_base64,
            "mime_type": "image/jpeg",
        },
    ]
} 



@traceable(run_type="llm")
def llm_raw(input):
    return output


llm_raw(input)


{'role': 'user',
 'content': [{'type': 'text', 'text': 'Describe the content of this image.'},
  {'type': 'image',
   'base64': '/9j/4QDeRXhpZgAASUkqAAgAAAAGABIBAwABAAAAAQAAABoBBQABAAAAVgAAABsBBQABAAAAXgAAACgBAwABAAAAAgAAABMCAwABAAAAAQAAAGmHBAABAAAAZgAAAAAAAABIAAAAAQAAAEgAAAABAAAABwAAkAcABAAAADAyMTABkQcABAAAAAECAwCGkgcAFgAAAMAAAAAAoAcABAAAADAxMDABoAMAAQAAAP//AAACoAQAAQAAAMgAAAADoAQAAQAAACwBAAAAAAAAQVNDSUkAAABQaWNzdW0gSUQ6IDIzN//bAEMACAYGBwYFCAcHBwkJCAoMFA0MCwsMGRITDxQdGh8eHRocHCAkLicgIiwjHBwoNyksMDE0NDQfJzk9ODI8LjM0Mv/bAEMBCQkJDAsMGA0NGDIhHCEyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMv/CABEIASwAyAMBIgACEQEDEQH/xAAbAAACAwEBAQAAAAAAAAAAAAADBAACBQEGB//EABgBAQEBAQEAAAAAAAAAAAAAAAABAgME/9oADAMBAAIQAxAAAAHRMQ3DqCpzAk9FQU51SWMK6IelhFws0BAdGL9M4iHNAAkwWq3VhAEcgRf5/n9MfRgfPZZ76eDLXt1fHQ9aXxtz37fzUmX0S/nPT4329+S2BagNdDx+8+mycXU3ne3FuctszLlviecnbjOdhXs6c5bhLVgWvIV2cbkfUSfN5jfu/LYlNZtXh9Q3rUtLl0PS9saVjUr5zyTvxkuQDL9KcK0IFfWXq7lUTh6gJzpaluHTM2FSLVNXQ8zeX2k8XMaGWs6YvBWohISAVCY

## Gap in image rendering 

We don't render images
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%2222select%22%3Atrue%2C%22id%22%3Atrue%2C%22status%22%3Atrue%2C%22name%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22start_time%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Atrue%2C%22last_queued_at%22%3Atrue%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Atrue%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22feedback_stats%22%3Atrue%2C%22reference_example_id%22%3Atrue%2C%22actions%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&peek=7c87a2bd-fa0e-4aa0-b8b3-2719852a8757&peeked_trace=7c87a2bd-fa0e-4aa0-b8b3-2719852a8757

In [ ]:
#images

import base64
import httpx

image_url = "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U"
image_base64 = base64.standard_b64encode(httpx.get(image_url).content).decode("utf-8")


input = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {"type": "image", "url": "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U"},
    ]
}

# From base64 data
output = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {
            "type": "image",
            "base64": image_base64,
            "mime_type": "image/jpeg",
        },
    ]
} 



@traceable(run_type="llm")
def llm_raw(input):
    return output


llm_raw(input)